<a href="https://colab.research.google.com/github/Qaladid/machine-learning-course/blob/main/homework_serverless.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import urllib

# Define the URL of the model
model_url = 'https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/model_2024_hairstyle.keras'
model_path = 'model_2024_hairstyle.keras'

# Download the model file
urllib.request.urlretrieve(model_url, model_path)
print("Model downloaded successfully!")

Model downloaded successfully!


In [2]:
import tensorflow as tf

# Load the Keras model
model = tf.keras.models.load_model(model_path)
print("Model loaded successfully!")

Model loaded successfully!


In [3]:
# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the converted model as a .tflite file
tflite_model_path = 'model_2024_hairstyle.tflite'
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

print("Model converted to TF Lite format and saved!")

Saved artifact at '/tmp/tmp2x74be9r'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 200, 200, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  140435729463280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140435728419824: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140435728430384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140435727911568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140435727914208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140435728212992: TensorSpec(shape=(), dtype=tf.resource, name=None)
Model converted to TF Lite format and saved!


In [4]:
import os

# Get the size of the converted model in MB
model_size = os.path.getsize(tflite_model_path) / (1024 * 1024)  # Size in MB
print(f"Size of the converted TF Lite model: {model_size:.2f} MB")

Size of the converted TF Lite model: 76.58 MB


In [5]:
import tensorflow as tf

# Load the TensorFlow Lite model
interpreter = tf.lite.Interpreter(model_path="model_2024_hairstyle.tflite")
interpreter.allocate_tensors()

# Get the input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Print the output details to check the index
print("Input details:", input_details)
print("Output details:", output_details)

# Get the index of the output
output_index = output_details[0]['index']
print(f"Output index: {output_index}")

Input details: [{'name': 'serving_default_input_layer:0', 'index': 0, 'shape': array([  1, 200, 200,   3], dtype=int32), 'shape_signature': array([ -1, 200, 200,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Output details: [{'name': 'StatefulPartitionedCall_1:0', 'index': 13, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Output index: 13


In [6]:
from io import BytesIO
from urllib import request
from PIL import Image
import numpy as np

# Download the image
def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img

# Prepare the image: resize and ensure RGB format
def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')  # Convert to RGB if not already
    img = img.resize(target_size, Image.NEAREST)  # Resize the image to target size
    return img

# URL of the image
image_url = "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"

# Download and prepare the image
img = download_image(image_url)

# Target size (we need to determine this)

In [7]:
# Assuming the target size is (200, 200)
target_size = (200, 200)
img_resized = prepare_image(img, target_size)

In [8]:
# Convert image to numpy array
img_array = np.array(img_resized)

# Normalize pixel values (if the model expects a range between 0 and 1)
img_array = img_array / 255.0

# Check the shape of the array
print(img_array.shape)  # This should be (200, 200, 3) if the target size is (200, 200)

(200, 200, 3)


In [9]:
# Get the Red channel value for the first pixel (at position [0, 0])
red_channel_value = img_array[0, 0, 0]  # [0, 0] is the first pixel, 0 is for the Red channel

print(f"Red channel value of the first pixel: {red_channel_value}")

Red channel value of the first pixel: 0.23921568627450981


In [10]:
!pip install tflite-runtime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 26.3 MB/s eta 0:00:00


In [11]:
import tensorflow as tf
import numpy as np

# Load the TensorFlow Lite model
model_path = '/content/model_2024_hairstyle.tflite'  # Update this with the correct path
interpreter = tf.lite.Interpreter(model_path=model_path)

# Allocate tensors
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Prepare the image: Ensure it is in the correct shape
# Assuming the target size is (200, 200)
img_resized = prepare_image(img, (200, 200))

# Convert the image to a numpy array and normalize it (same steps as before)
img_array = np.array(img_resized) / 255.0

# Reshape to match the model input (if necessary, sometimes it requires adding a batch dimension)
img_array = np.expand_dims(img_array, axis=0)  # Shape becomes (1, 200, 200, 3)

# Set the input tensor
input_tensor_index = input_details[0]['index']
interpreter.set_tensor(input_tensor_index, img_array.astype(np.float32))

# Run inference
interpreter.invoke()

# Get the output tensor
output_tensor_index = output_details[0]['index']
output = interpreter.get_tensor(output_tensor_index)

# Print the output prediction
print("Model Output:", output[0][0])  # The model returns the prediction

Model Output: 0.8937741
